# 단어 빈도를 이용한 벡터화

## 벡터화
- 정의: 전처리 과정에서 텍스트를 숫자로 변환하는 과정
- 목적 : 기계가 자연어 처리를 할 수 있도록 데이터를 변환하는 과정
- 방법
  1. 통계와 머신 러닝을 활용한 방법 (오늘의 학습 범위)
  2. 인공 신경망을 활용하는 방법

## Bag of Words (BoW) 정의
- 정의 : 문서를 단어들의 집합으로 보고, 각 단어의 빈도수를 기반으로 문서를 벡터화하는 기법
- 특징
  - 문서를 단어 기준으로 벡터화
  - 단어 순서나 문맥은 무시
  - 오직 단어의 존재 여부 또는 등장 빈도만 고려
  - 단순하지만 강력한 표현 방식으로 기계학습에 자주 사용됨
  - 문서 간 비교, 분류, 군집화 등의 기초적인 텍스트 분석에 유용

## BoW 구현하기

### Keras Tokenizer 활용
 - 단어장 : 중복을 제거한 단어들의 집합
 - 단어장에는 단어의 "등장 여부"만 나타나고, 등장 "횟수"는 나타나지 않음.
 - BoW는 단어장을 바탕으로 등장 횟수까지 계산한 벡터 표현임.


In [42]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print("Bag of Words :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


### sckit-learn Countvectorizer 활용

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


In [44]:
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))

단어장(Vocabulary)의 크기 : 10


## DTM과 코사인 유사도

### DTM (Document-Term Matrix, 문서-단어 행렬)
 - 여러 문서의 **bag of words**를 하나의 행렬로 구현한 것
 - 행: 문서 / 열: 단어
 - 각 원소는 해당 문서에서 특정 단어가 등장한 빈도수

    예: DTM[문서1, 단어3] = 2 → 문서1에서 단어3이 2번 등장
    
💡 **TDM(Term-Document Matrix)**는 반대 구조: 행 = 단어, 열 = 문서



### 코사인 유사도(Cosine Similarity)
- 두 벡터(문서 간)의 방향이 얼마나 유사한지를 측정
- 공식: 코사인 각도 → 값의 범위 -1 ~ 1
  - 1 → 방향이 동일 → 매우 유사
  - 0 → 직각 → 유사도 없음
  - -1 → 반대 방향 → 완전히 다름

🔍 텍스트 유사도 분석, 추천 시스템, 이미지 비교 등에 활용

In [45]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [46]:
print('{:.2f}'.format(cos_sim(doc1, doc2))) #문서1과 문서2의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc1, doc3))) #문서1과 문서3의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc2, doc3))) #문서2과 문서3의 코사인 유사도

0.67
0.67
1.00


- 문서3은 문서2에서 단지 모든 단어의 빈도수가 1씩 증가함.
- 다시 말해 한 문서 내의 모든 단어의 빈도수가 동일하게 증가하는 경우에는 기존의 문서와 코사인 유사도의 값이 1이라는 것

📌 한줄 요약

DTM은 문서별 단어 빈도를 행렬로 표현한 것이고, 코사인 유사도는 두 문서 간 벡터의 방향 유사도를 수치로 나타내는 방법입니다.

## DTM 구현과 문제점

### scikit-learn CountVectorizer 활용

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


### DTM의 한계점
1. DTM은 대부분 0으로 채워진 희소 행렬이 되어 저장 공간과 계산 효율에 문제가 생김.

2. 단순 빈도 기반 DTM은 중요하지 않은 단어(예: 'the')에도 동일한 가중치를 주어 문서 유사도 판단에 한계가 발생함.

## TF-IDF (Term Frequency - Inverse Document Frequency)
- 정의: 단어의 중요도를 계산하기 위한 가중치 기법
- 목적: 모든 문서에 자주 등장하는 중요하지 않은 단어(예: the, is 등)의 영향은 줄이고, 특정 문서에만 자주 등장하는 단어의 중요도를 강조
- 주요 특징
  - TF(단어 빈도): 해당 문서에서 단어가 얼마나 자주 등장했는지
  - IDF(역문서 빈도): 그 단어가 전체 문서 중 얼마나 희귀한지
  - TF × IDF: 자주 등장하면서도 희귀한 단어에 높은 가중치 부여
  - 노이즈 완화: 불용어처럼 모든 문서에 자주 등장하는 단어의 영향 감소
  - DTM 기반: 먼저 DTM을 만든 후, 각 값에 TF-IDF 가중치를 곱해 적용
  - 단점: 항상 DTM보다 성능이 좋은 것은 아님 (문맥 정보 없음)
- 📌 한줄 요약
  - TF-IDF는 단어의 빈도와 희귀도를 반영해 문서 내 단어의 중요도를 계산하며,
  - DTM 기반으로 가중치를 부여해 불필요한 단어의 영향을 줄이는 기법입니다.

## TF-IDF 구현하기

### 수식으로 TF-IDF 구현하기

- 도구 import

In [48]:
from math import log
import pandas as pd
print('=3')

=3


- 대상 문서

In [49]:
docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',
]
print('=3')

=3


- 단어장 생성 (DTM의  열)

In [50]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [51]:
N = len(docs) # 총 문서의 수
N

3

In [52]:
def tf(t, d): #TF 구하는 함수
    return d.count(t)

def idf(t): #IDF 구하는 함수
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1)) + 1

def tfidf(t, d): #TF-IDF 함수
    return tf(t,d)* idf(t)

- DTM 만들기 : TF 함수 활용

In [53]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


- 각 단어의 IDF

In [54]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns=["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


- TF-IDF 행렬 출력

In [55]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


### scikit-learn TFidVectorizer 활용하여 TF-IDF 구현하기

- 다른점 : TfidfVectorizer는 파이썬으로 구현한 식에서 log 항의 분자에도 1을 더해주며, TF-IDF의 결과에 L2 Norm까지 추가로 수행함

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns = vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


# LSA와 LDA

## LSA(Latent Semantic Analysis)
 - 정의 : 전체 코퍼스에서 문서 속 단어들 사이의 관계를 찾아내는 자연어 처리 정보 검색 기술
 - 단어와 단어 사이, 문서와 문서 사이, 단어와 문서 사이의 의미적 유사성 점수를 찾아낼 수 있음
 - 어떤 문서에서 특정 단어들의 빈도가 몇인지를 판단하는 것보다 효과적인 경우가 많음

### 특이값 분해
- 고유벡터: 어떤 행렬에 의해 방향은 안 변하고 크기만 변하는 벡터
- 고유값: 그 크기가 몇 배로 변했는지를 나타내는 값
- 단위 행렬 : 주대각 성분이 모두 1이며 나머지 성분은 모두 0인 정사각행렬
- 역행렬 : n차 정사각행렬 행렬 A에 대해 어떤 행렬을 곱했을 때, 결과 행렬이 단위 행렬이 되게하는 행렬
- 특이값 분해(특이분해, singular-decomposition)
  - m × n 크기의 임의의 사각 행렬 A를 아래 3개의 행렬의 곱으로 나타내는 것

   A = U * Σ * V

   - 대각성분이 양수인 대각행렬이어야 한다. 큰 수부터 작은 수 순서로 배열한다.
   - U 는 N 차원 정방행렬로 모든 열벡터가 단위벡터이고 서로 직교해야 한다.
   - V 는 M 차원 정방행렬로 모든 열벡터가 단위벡터이고 서로 직교해야 한다.

  - 위 조건을 만족하는 행렬
의 대각성분들을 특잇값(singular value), 행렬
의 열벡터들을 왼쪽 특이벡터(left singular vector), 행렬
의 행벡터들을 오른쪽 특이벡터(right singular vector)라고 부른다.

- [정리] 특이분해는 모든 행렬에 대해 가능하다. 즉 어떤 행렬이 주어지더라도 위와 같이 특이분해할 수 있다.

- 특이값 분해 행렬의 크기 : 특잇값의 개수는 행렬의 열과 행의 개수 중 작은 값과 같다.

- 특이값 분해의 축소형 : 특잇값 대각행렬에서 0인 부분은 사실상 아무런 의미가 없기 때문에 대각행렬의 0 원소 부분과 이에 대응하는 왼쪽(혹은 오른쪽) 특이벡터들을 없애고 다음처럼 축소된 형태로 해도 마찬가지로 원래 행렬이 나온다.

- Truncated SVC(절단된 특이값 분해)
  - 특이값 가운데 가장 큰(가장 중요한) t개만 남기고 해당 특이값에 대응되는 특이 벡터(sigular vector)들로 행렬 A를 근사(approxiamte)한 결과
  - 수행 시, 행렬 Σ의 대각 원소값 중에서 상위값 t개만 남게되며, U행렬과 V 행렬의 t까지 남음.
  - 세 행렬에서 값의 손실이 일어나 기존의 행렬 A를 정확히 복구할 수는 없게 됨.
  - t = 하이퍼파라미터, 클수록 다양한 의미를 줄 수 있으나, 노이즈를 제거하려면 t를 잡게 잡아야함.

- LSA는 DTM이나 TF-IDF 행렬 등에 Truncated SVD를 수행합니다. 이렇게 분해하여 얻은 행렬 3개(Uk,VkT,S)는 각각 '문서들과 관련된 의미들을 표현한 행렬', '단어들과 관련된 의미를 표현한 행렬' , '각 의미의 중요도를 표현한 행렬' 이라고 해석할 수 있음.

## LSA 실습

- 라이브러리 불러오기

In [57]:
#import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
print('=3')

=3


In [58]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# 이거 추가로 다운로드해야함
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

- 데이터 불러오기

In [59]:
import os

## Colab에서는 이렇게 바꿔주시면됩니다

# csv_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/abcnews-date-text.csv'
csv_filename = './abcnews-date-text.csv'

# 파일 다운로드
url = "https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv"
urllib.request.urlretrieve(url, filename=csv_filename)

('./abcnews-date-text.csv', <http.client.HTTPMessage at 0x7b813a4d47d0>)

In [60]:
data = pd.read_csv(csv_filename, on_bad_lines='skip')
data.shape

(1082168, 2)

- 데이터 확인하기

In [61]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [62]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [63]:
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

,0
headline_text,1054983


In [64]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

### 데이터 정제 및 정규화

In [65]:
import nltk


In [66]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [67]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


### 역토큰화 및 DTM 생성

In [68]:
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc
print('=3')

=3


In [69]:
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [70]:
# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)
print('=3')

=3


In [71]:
# DTM의 크기(shape)는 (문서의 수 × 단어 집합의 크기)
print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


### scikit-learn TruncatedSVD 활용

In [72]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 0.01202873, -0.00364392,  0.0182708 , ...,  0.00370654,
        -0.00219778,  0.01428444],
       [ 0.02902364, -0.01088561,  0.01816535, ..., -0.00416315,
         0.00910237, -0.00564994],
       [ 0.0050253 , -0.00199274,  0.00971996, ..., -0.00260395,
        -0.00294113,  0.00173509],
       ...,
       [ 0.02969623,  0.00442217,  0.02515941, ...,  0.03093247,
        -0.00776201,  0.02137486],
       [ 0.06149513, -0.00512111,  0.13866209, ...,  0.9953926 ,
        -0.62953432, -0.50654974],
       [ 0.07167623,  0.02841163,  0.00114059, ..., -0.0080282 ,
         0.0136475 ,  0.07459133]])

In [73]:
print(lsa_model.components_.shape)

(10, 5000)


In [74]:
terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', np.float64(0.7469)), ('man', np.float64(0.45379)), ('charge', np.float64(0.21103)), ('new', np.float64(0.14081)), ('court', np.float64(0.11142))]
Topic 2: [('man', np.float64(0.69442)), ('charge', np.float64(0.30023)), ('court', np.float64(0.16846)), ('face', np.float64(0.11403)), ('murder', np.float64(0.10644))]
Topic 3: [('new', np.float64(0.83787)), ('plan', np.float64(0.23635)), ('say', np.float64(0.18296)), ('govt', np.float64(0.10986)), ('council', np.float64(0.10861))]
Topic 4: [('say', np.float64(0.7389)), ('plan', np.float64(0.35987)), ('govt', np.float64(0.16727)), ('council', np.float64(0.12926)), ('urge', np.float64(0.07472))]
Topic 5: [('plan', np.float64(0.73579)), ('council', np.float64(0.17578)), ('govt', np.float64(0.13115)), ('urge', np.float64(0.08717)), ('water', np.float64(0.07321))]
Topic 6: [('govt', np.float64(0.54195)), ('court', np.float64(0.28657)), ('urge', np.float64(0.21961)), ('fund', np.float64(0.20015)), ('face', np.float64(0.15983)

- 토픽 모델링(Topic Modelling)
  - 문서의 집합에서 토픽을 찾아내는 프로세스
  -  고객의 소리와 같이 많은 문서에서 주요 주제를 알아내는 일이 중요할 때 사용

## LDA (Latent Dirichlet Allocation, LDA)


### 개요
- 서들이 토픽들의 혼합으로 구성되어 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정합니다. 그리고 데이터가 주어지면, LDA는 이 가정에 따라 단어들의 분포로부터 문서가 생성되는 과정을 역추적해 문서의 토픽을 찾아냅니다.

-  각 토픽의 단어 분포, 즉 특정 토픽에 특정 단어가 나타날 확률을 추정

### LDA의 가정
- 아래의 과정을 통해 문서가 작성되었다는 가정 하에 LDA는 토픽을 뽑아내기 위하여 위 과정을 역으로 추적하는 역공학(reverse engneering)을 수행합니다.
1. 문서에 사용할 단어의 개수 N을 정합니다.
2. 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정합니다.
3. 문서에 사용할 각 단어를 정합니다.
  3-1. 토픽 분포에서 토픽 T를 확률적으로 고릅니다.
  3-2. 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고릅니다.



### LDA의 수행하기
1. 사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.
앞서 말하였듯이 LDA에게 토픽의 개수를 알려주는 역할은 사용자의 역할입니다. LDA는 토픽의 개수 k를 입력받으면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정합니다.

2. 모든 단어를 k개 중 하나의 토픽에 할당합니다.
이제 LDA는 모든 문서의 모든 단어에 대해서 k개 중 하나의 토픽을 랜덤으로 할당합니다. 이 작업이 끝나면 각 문서는 토픽을 가지며, 토픽은 단어 분포를 가지는 상태입니다. 물론 랜덤으로 할당하였기 때문에 사실 이 결과는 전부 틀린 상태입니다. 만약 한 단어가 한 문서에서 2회 이상 등장하였다면, 각 단어는 서로 다른 토픽에 할당되었을 수도 있습니다.

3. 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행합니다. (iterative)
  3-1. 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당됩니다.
   - p(topic t | document d) : 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
   - p(word w | topic t) : 각 토픽들 t에서 해당 단어 w의 분포

### 잠재 디리클레 할당(LDA)과 잠재 의미 분석(LSA)의 차이
- LSA : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.
- LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

## LDA 실습

### TF-IDF 행렬 생성

In [75]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


### scikit-learn LDA Model 활용

In [76]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.69841093, 0.0335099 ,
        0.0335099 ],
       [0.03365628, 0.03365628, 0.03365628, ..., 0.03365628, 0.03365628,
        0.03365628],
       [0.03661873, 0.0366096 , 0.0366096 , ..., 0.67050448, 0.0366096 ,
        0.0366096 ],
       ...,
       [0.02914502, 0.14077174, 0.16177343, ..., 0.3678171 , 0.02914502,
        0.02914502],
       [0.02637829, 0.13847841, 0.02637829, ..., 0.02637829, 0.02637829,
        0.12325015],
       [0.22554418, 0.03376055, 0.03376055, ..., 0.03376055, 0.21169868,
        0.03376055]])

In [77]:
print(lda_model.components_.shape)

(10, 5000)


In [78]:
# LDA의 결과 토픽과 각 단어의 비중을 출력합시다
terms = tfidf_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(lda_model.components_, terms)


Topic 1: [('new', np.float64(6312.18956)), ('home', np.float64(4047.88866)), ('hit', np.float64(3572.56771)), ('ban', np.float64(2995.55782)), ('market', np.float64(2922.48643))]
Topic 2: [('australia', np.float64(9367.76267)), ('charge', np.float64(5947.44682)), ('sydney', np.float64(5850.75816)), ('man', np.float64(5681.84494)), ('murder', np.float64(4677.14748))]
Topic 3: [('say', np.float64(7586.74762)), ('woman', np.float64(3985.7297)), ('court', np.float64(3868.11592)), ('open', np.float64(3770.65617)), ('face', np.float64(3612.76986))]
Topic 4: [('police', np.float64(6175.26407)), ('queensland', np.float64(5552.6178)), ('coast', np.float64(3825.45613)), ('tasmanian', np.float64(3550.61525)), ('gold', np.float64(2674.6521))]
Topic 5: [('melbourne', np.float64(5298.43132)), ('school', np.float64(3966.56664)), ('2016', np.float64(3962.51293)), ('rural', np.float64(3521.89517)), ('warn', np.float64(3379.28397))]
Topic 6: [('australian', np.float64(7674.55731)), ('perth', np.float64(

# 텍스트 분포를 이용한 비지도 학습 토크나이저

## 형태소 분석기와 단어 미등록 문제

### 형태소 분석기의 필요성
- 한국어는 교착어
- 교착어란 하나의 낱말(어절)이 하나의 어근(root, 혹은 어간 stem)과 각각 단일한 기능을 가지는 하나 이상의 접사(affix)의 결합으로 이루어져 있는 언어

In [79]:

en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

['The', 'dog', 'ran', 'back', 'to', 'the', 'corner', 'near', 'the', 'spare', 'bedrooms']


In [80]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


In [81]:
# 이거 다운로드 받아주셔야 됩니다!
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 31.7 MB/s eta 0:00:00


In [82]:
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


### 단어 미등록 문제
 - 해결 방법
   - 사용자 사전 등록
     - 사용자가 직접 미등록 단어를 사전에 등록해 주는 방법
     - 미등록 단어에 대한 분석 정확도 향상
   - 자동 사전 생성
     - 미등록 단어를 자동으로 추출하여 사전에 추출하는 방법
     - 대용량 텍스트를 활용하여 빈도 수 등을 기반으로 자동으로 사전을 생성
   - 기존 단어 활용
     - 기존 어간과 유사한 부분이 있다면, 해당 부분을 활용하여 분석을 수행
   - 문맥 분석
     - 미등록 단어의 문맥을 분석하여, 해당 단어가 명사, 동사 등의 품사 중 어떤 것에 가까운지를 예측하여 분석 수행
     - 머신 러닝 기반의 분류 모델을 이용하여 수행

In [83]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


## soynlp
 - 품사 태깅, 형태소 분석 등을 지원하는 한국어 형태소 분석기
 - 비지도 학습으로 형태소 분석을 한다
 - 데이터에 자주 등장하는 단어들을 형태소로 분석합니다
 -  내부적으로 단어 점수표로 동작
 - 응집 확률(cohesion probability) 과 브랜칭 엔트로피(branching entropy) 를 활용


In [84]:
import urllib.request

# txt_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/2016-10-20.txt'
txt_filename = './2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

('./2016-10-20.txt', <http.client.HTTPMessage at 0x7b8159e79ad0>)

In [85]:
# 이거 다운로드 받아주셔야 됩니다!
!pip install soynlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 5.7 MB/s eta 0:00:00


In [86]:
from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus(txt_filename)
len(corpus)

30091

In [87]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

In [88]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 3.869 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


### soynlp의 응집확률
- 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
- 문자열을 문자 단위로 분리하여 내부 문자열을 만드는 과정에서, 왼쪽부터 순서대로 문자를 추가하면서 각 문자열이 주어졌을 때 그다음 문자가 나올 확률을 계산하여 누적 곱을 한 값
- 이 값이 높을수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높음

In [89]:
word_score_table["반포한"].cohesion_forward

np.float64(0.08838002913645132)

In [90]:
word_score_table["반포한강"].cohesion_forward

np.float64(0.19841268168224552)

In [91]:
word_score_table["반포한강공"].cohesion_forward

np.float64(0.2972877884078849)

In [92]:
word_score_table["반포한강공원"].cohesion_forward

np.float64(0.37891487632839754)

In [93]:
word_score_table["반포한강공원에"].cohesion_forward

np.float64(0.33492963377557666)

### soynlp의 브랜칭 엔트로피(branching entropy)
-  확률 분포의 엔트로피값을 사용
-  주어진 문자열에서 다음 문자가 등장할 수 있는 가능성을 판단하는 척도

In [94]:
word_score_table["디스"].right_branching_entropy

1.6371694761537934

In [95]:
word_score_table["디스플"].right_branching_entropy

-0.0

In [96]:
word_score_table["디스플레"].right_branching_entropy

-0.0

In [97]:
word_score_table["디스플레이"].right_branching_entropy

3.1400392861792916

### soynlp의 LTokenizer

In [100]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

### 최대 점수 토크나이저

In [99]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

# 종합 요약
- DTM(Document Term Matrix)  
  - 문서-단어 행렬
  - 각 문서에서 사용된 단어의 빈도를 표시
  - 행은 문서를 나타내고 열은 단어를 나타냄
  -  텍스트 데이터의 각 문서를 벡터로 표현할 수 있으므로, 이를 활용해 문서 간 유사도를 계산하거나 토픽 모델링 등의 분석을 수행할 수 있음

- TF-IDF (Term Frequency-Inverse Document Frequency)
  - DTM에서 특정 단어가 얼마나 중요한지를 나타내는 지표
  - 많이 나오는 일반적인 단어들은 중요하지 않은 단어로 간주되고, 문서를 잘 나타내는 중요한 단어들이 높은 점수를 받게 됨

- LSA (Latent Semantic Analysis)
  - DTM을 활용해 문서 간 유사도를 계산하는 방법 중 하나.
  - 단어-문서 행렬에서 잠재적인 의미를 추출하여 문서를 벡터 공간 상에 표현
  - 문서 간 유사도를 계산할 수 있음

- LDA (Latent Dirichlet Allocation)  
  - 문서 내의 토픽 분포와 토픽 내 단어 분포를 추정하여 토픽 모델링을 수행하는 방법
  - 각 문서는 여러 개의 토픽으로 구성
  - 각 토픽은 단어들의 확률 분포로 나타남.
  0 각 문서가 어떤 주제를 다루는지를 파악할 수 있음

- soynlp
 - 한국어 자연어 처리를 위한 파이썬 라이브러리
 -  자주 등장하는 신조어나 오탈자, 그리고 형태소 분석에서 발생하는 OOV(Out-of-Vocabulary) 문제를 처리하는 데 강점
 - 데이터에서 단어 빈도수를 기반으로 단어들을 자동으로 추출하며, 이를 활용해 형태소 분석을 수행